In [ ]:
# 级联删除数据库下的表
drop database if exists mydb cascade;

create table employee(
 id int
 ,name string
 ,hobby array<string>
 ,add map<String,string>
)
row format delimited fields terminated by ','
collection items terminated by '-'
map keys terminated by ':';

# 查看基础表结构
desc employee
# 查看详细的表结构信息
desc formatted employee;
# 查看建表语句
show create table employee;

# 查看数据库的描述信息和⽂件⽬录位置路径信息；
desc database mydatabase;

# 查看表是否是管理表或者是外部表  tableType:MANAGED_TABLE 表示管理表  tableType:EXTERNAL_TABLE 表示外部表
desc extended  table_name 

***********************************************************************************************************************************
create table t1(
 id int
 ,name string
 ,hobby array<string>
 ,add map<String,string>
)
row format delimited fields terminated by ','
collection items terminated by '-'
map keys terminated by ':';

# overwrite是覆盖，把overwrite换成into是追加。 
load data local inpath '/home/hadoop/Desktop/data' overwrite into table t1;
load data local inpath '/usr/host/data'  into table testtable;

# 加载本地数据
load data local inpath 'localpath' [overwrite] into table tablename 
# 加载hdfs数据
load data inpath 'hdfspath'[overwrite] into table tablename


create external table t2(
 id int
 ,name string
 ,hobby array<string>
 ,add map<String,string>
)
row format delimited fields terminated by ','
collection items terminated by '-'
map keys terminated by ':'
location '/user/t2';

load data local inpath '/home/hadoop/Desktop/data' overwrite into table t2;


***********************************************************************************************************************************
create table student_tmp(id INT, name STRING)
partitioned by(academy STRING, class STRING)
row format delimited fields terminated by ',';

load data local inpath '/Users/jack/Documents/data' into table student_tmp partition(academy='physics',class='034'); 

# 查看表的所有分区：
show partitions student_tmp
show partitions employees1 partition(country='china')

***********************************************************************************************************************************
create table employees
 (
 name string,
 salary float,
 subordinated array<string>,
 deductions map<string,float>,
 address struct<street:string,city:string,state:string,zip:int>
 )
 partitioned by (country string,state string)
 row format delimited fields terminated by '\001'
 collection items terminated by '\002'
 map keys terminated by '\003'
 lines terminated by "\n"
 stored as textfile; # 默认

# 如果⽂件数据是纯⽂本，可以使⽤ stored as textfile。如果数据需要压缩，使⽤ stored as sequencefile 。

# 分区操作
alter table employees add partition (country="US",state="CA");
alter table employees drop partition(country="china",state="Asia");

# 1，把⼀个分区打包成⼀个har包
 alter table employees archive partition (country="china",state="Asia")
# 2, 把⼀个分区har包还原成原来的分区
 alter table employees unarchive partition (country="china",state="Asia")
    
# 3, 保护分区防⽌被删除
 alter table employees partition (country="china",state="Asia") enable no_drop
# 4,保护分区防⽌被查询
 alter table employees partition (country="china",state="Asia") enable offline

# 5，允许分区删除和查询
 alter table employees partition (country="china",state="Asia") disable no_drop
 alter table employees partition (country="china",state="Asia") disable offline

insert overwrite/into table copy_employees partition（country="china",state="Asia"） 
select * from employees es where es.country="china" and es.state ="Asia"

# 修改表名
alter table employees rename to employees_bak;


# 外部表
# 添加分区表
alter table logmsgs add if not exists
partition(year=2017,month=1,day=1) location 'logs/2017/01/01'
partition(year=2017,month=1,day=2) location 'logs/2017/01/02'
# 修改分区表
alter table logmsgs partition(year=2017,month=02,day=09) set location '/usr/log/2017/02/09'
# 增加列的信息
alter table logmsgs add columns(app_name string comment 'Application name',session_id long comment 'The current session id');
# 修改列的信息
alter table logmsgs
change column hms hours_minutes_seconds int
comment 'The hours,minutes and seconds part of the timestamp'
after severity;

# 创建数据库，判断该数据库是否存在，并给数据库⼀个基本描述
CREATE DATABASE IF NOT EXISTS hive2
COMMENT "this is ruoze database"
WITH DBPROPERTIES ("creator"="ruoze", "date"="2018-08-08");


# Hive不⽀持⽤insert语句⼀条⼀条的进⾏插⼊操作，也不⽀持update操作。数据是以load的⽅式加载到建⽴好的表中。数据⼀旦导⼊就不可以修改。

# ORDER BY与SORT BY的不同
# ORDER BY 全局排序，只有⼀个Reduce任务;SORT BY 只在本机做排序


# 查询数组类型,STRUCT类型,MAP类型
SELECT name,subordinates[0],address.city,deductions["State Taxes"] FROM employees

***********************************************************************************************************************************
hive> create table bucketed_user(
    > id string,
    > name string
    > )
    > clustered by(id) sorted by(name) into 4 buckets
    > row format delimited fields terminated by '\t' lines terminated by '\n'
    > stored as textfile;

insert overwrite table bucketed_user select name,addr from testtext;

alter table tablename change column c1 c2 int
# 修改分隔符
alter table city set serdeproperties('field.delim'='\t');
# 修改location 
alter table city set location 'hdfs://hadoop1:9000/location';
# 内部表转外部表
alter table table_name set TBLPROPERTIES('EXTERNAL'='TRUE');
# 外部表转内部表
alter table table_name set TBLPROPERTIES('EXTERNAL'='FALSE');





In [ ]:
select 1 from lxw_dual where 'football' like 'foot%';
select 1 from lxw_dual where 'football' like 'foot____';
select 1 from lxw_dual where NOT 'football' like 'fff%';

# JAVA 的 LIKE 操作
select 1 from lxw_dual where 'footbar’ rlike '^f.*r$’;
# 判断一个字符串是否全为数字
select 1 from lxw_dual where '123456' rlike '^\\d+$';
select 1 from lxw_dual where '123456aa' rlike '^\\d+$';

# 时间戳转日期
select from_unixtime(1323308943,'yyyyMMdd') from lxw_dual;
# 日期转时间戳
select unix_timestamp('20111207 13:01:03','yyyyMMdd HH:mm:ss') from lxw_dual;

select datediff('2012-12-08','2012-05-09') from lxw_dual;
select date_add('2012-12-08',10) from lxw_dual;
select date_sub('2012-12-08',10) from lxw_dual;

select if(1=2,100,200) from lxw_dual;
select COALESCE(null,'100','50') from lxw_dual;  # 100

select reverse(abcedfg’) from lxw_dual;  # gfdecba
select concat(‘abc’,'def’,'gh’) from lxw_dual;  # abcdefgh
select concat_ws(',','abc','def','gh') from lxw_dual;  # abc,def,gh
select substring('abcde',-2,2) from lxw_dual;  # de
select regexp_replace('foobar', 'oo|ar', '') from lxw_dual;  # fb
select regexp_extract('foothebar', 'foo(.*?)(bar)', 2) from lxw_dual;  # bar
select get_json_object(request_uri_dmalog_show_post_id,'$.pn') from ods_rtm_app_ev 

select space(10) from lxw_dual;  # 空格字符串函数
select repeat('abc',5) from lxw_dual;  # abcabcabcabcabc
select lpad('abc',10,'td') from lxw_dual;  # tdtdtdtabc
select rpad('abc',10,'td') from lxw_dual;  # abctdtdtdt
select split('abtcdtef','t') from lxw_dual;  # ["ab","cd","ef"]

# 集合查找函数:  find_in_set(string str, string strList)
# 说明: 返回 str 在 strlist 第一次出现的位置，strlist 是用逗号分割的字符串。如果没有找到该 str 字符，则返回 0
hive> select find_in_set('ab','ef,ab,de') from lxw_dual;
2
hive> select find_in_set('at','ef,ab,de') from lxw_dual;
0


hive> create table lxw_test as select array("tom","mary","tim") as t from lxw_dual;
hive> select t[0],t[1],t[2] from lxw_test;
tom mary tim

hive> Create table lxw_test as select map('100','tom','200','mary') as t from lxw_dual;
hive> select t['200'],t['100'] from lxw_test;
mary tom

hive> create table lxw_test as select struct('tom','mary','tim') as t from lxw_dual;
hive> describe lxw_test;
t struct<col1:string,col2:string,col3:string>
hive> select t.col1,t.col3 from lxw_test;
tom tim  


# Hive
# Hive中实现group_concat功能（不用udf） 
# 注意:collect_set 只能返回不重复的集合 若要返回带重复的要用collect_list
SELECT id,concat_ws(',', collect_set(name)) FROM t GROUP BY id;









In [ ]:
hadoop fs -put [local 文件] [目标目录]
hadoop fs -put xielaoshi /data/



sss

In [ ]:
# Hive分区和分桶区别
1.分桶随机分割数据库，分区是非随机分割数据库。因为分桶是按照列的哈希函数进行分割的，相对比较平均；
  而分区是按照列的值来进行分割的，容易造成数据倾斜。
2.分桶是对应不同的文件（细粒度），分区是对应不同的文件夹（粗粒度）。
  桶是更为细粒度的数据范围划分，分桶的比分区获得更高的查询处理效率，使取样更高效。
3.注意：普通表（外部表、内部表）、分区表这三个都是对应HDFS上的目录，桶表对应是目录里的文件

get_json_object(userinfo,'$.name') 
json_tuple(userinfo,'name','age')

select id,b.name,b.age,b.sex
from default.userinfo
lateral view json_tuple(userinfo,'name','age','sex') b as name,age,sex;